In [84]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

In [85]:
h = np.load('./scripts and data/h.npy', allow_pickle=True).item().get('h').reshape(-1)
# there might be a better way to read the .wav file
_, wavin = wavfile.read('./scripts and data/myfile.wav')

In [86]:
def make_mp3_analysisfb(h: np.ndarray, M: int) -> np.ndarray:
    """
    """
    H = np.zeros([len(h), M], dtype=np.float32)
    for i in range(1, M + 1):
        n = np.arange(h.shape[0], dtype=np.int64)
        freq_i = (2 * i - 1) * np.pi / (2.0 * M)
        phas_i = -(2 * i - 1) * np.pi / 4.0
        tmp = np.cos(freq_i * n + phas_i)
        x = h * tmp
        H[:, i - 1] = x
    return H

def make_mp3_synthesisfb(h: np.ndarray, M: int) -> np.ndarray:
    """
    """
    H = make_mp3_analysisfb(h, M)
    L = len(h)
    G = np.flip(H, axis=0)
    return G

In [87]:
M = 32; L = 512; N = 36
H = make_mp3_analysisfb(h, M)
G = make_mp3_synthesisfb(h, M)
H.shape, G.shape

((512, 32), (512, 32))

In [95]:
def v(k, n):
        conv = 0
        for m in range(L):
            conv += wavin[n - m] * H[m, k]
        return conv

def y(k, n):
    return v(k, M*n)